In [69]:
import torch
import torchvision.utils as vutils
import numpy as np
import torchvision.models as models
from torchvision import datasets
from tensorboardX import SummaryWriter
import datetime
writer = SummaryWriter()
#有以下可用的方法
list(filter(lambda x:False if x.startswith("_") else True ,dir(writer)))

['add_audio',
 'add_custom_scalars',
 'add_custom_scalars_marginchart',
 'add_custom_scalars_multilinechart',
 'add_embedding',
 'add_figure',
 'add_graph',
 'add_histogram',
 'add_image',
 'add_image_with_boxes',
 'add_images',
 'add_onnx_graph',
 'add_pr_curve',
 'add_pr_curve_raw',
 'add_scalar',
 'add_scalars',
 'add_text',
 'add_video',
 'all_writers',
 'caffe2_enabled',
 'close',
 'default_bins',
 'export_scalars_to_json',
 'file_writer',
 'log_dir',
 'scalar_dict']

### 添加标量

In [13]:
#二十个step对应的损失
losses = [1.3,1.23,1.19,1.1,1.098,1.02,0.98,0.87,0.67,0.43,0.23,0.12,0.12,0.109,0.09,0.0009,0.005,0.0006,0.00064,0.00006]
for step in range(20):
    #add_scalar方法将信息写入日志文件
    writer.add_scalar("data/loss",losses[step],step)

### 可视化图片

In [51]:
#数据加载
from torchvision.datasets import ImageFolder
from torchvision import transforms,utils,models,datasets
simple_transform = transforms.Compose([transforms.Resize((120,120))
                                       ,transforms.ToTensor()
                                       #,transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                      ])
train_data = ImageFolder('../chapter4/DogVsCatData/train',simple_transform)
to_image = transforms.ToPILImage()
for i in range(8):
    writer.add_image("data/Dog_Cat_Image_{}".format(i),train_data[i][0],i)
    writer.add_image_with_boxes('data/Dog_Cat_Image_with_box_{}'.format(i), train_data[i][0], 
                                torch.Tensor([[10, 10,100, 100]]), i)

### 添加音频

In [52]:
for i in range(3):
    writer.add_audio("audio/audio_{}".format(i),train_data[i][0].view(-1),i)

### 普通文本

In [55]:
for i in range(2):
    writer.add_text('Text/Simple_Text', 'text logged at step:{}'.format(i), i)
    writer.add_text('Text/Markdown_Text', '''a|b\n-|-\nc|d''',i)

### PR精度召回率曲线

In [100]:
writer.add_pr_curve('精度召回率曲线', np.random.randint(2, size=100), np.random.rand(100), 10)

In [98]:
true_positive_counts = [75, 64, 21, 5, 0]
false_positive_counts = [150, 105, 18, 0, 0]
true_negative_counts = [0, 45, 132, 150, 150]
false_negative_counts = [0, 11, 54, 70, 75]
precision = [0.3333333, 0.3786982, 0.5384616, 1.0, 0.0]
recall = [1.0, 0.8533334, 0.28, 0.0666667, 0.0]

writer.add_pr_curve_raw('精度召回率曲线原始数据', true_positive_counts,
                                false_positive_counts,
                                true_negative_counts,
                                false_negative_counts,
                                precision,
                                recall, 10)

In [99]:
help(writer.add_pr_curve)

Help on method add_pr_curve in module tensorboardX.writer:

add_pr_curve(tag, labels, predictions, global_step=None, num_thresholds=127, weights=None, walltime=None) method of tensorboardX.writer.SummaryWriter instance
    Adds precision recall curve.
    
    Args:
        tag (string): Data identifier
        labels (torch.Tensor, numpy.array, or string/blobname): Ground truth data. Binary label for each element.
        predictions (torch.Tensor, numpy.array, or string/blobname):
        The probability that an element be classified as true. Value should in [0, 1]
        global_step (int): Global step value to record
        num_thresholds (int): Number of thresholds used to draw the curve.
        walltime (float): Optional override default walltime (time.time()) of event



### 导出数据到json文件，以便外部程序处理

In [64]:
writer.export_scalars_to_json("./all_scalars.json")

### 降维投影

In [97]:
dataset = datasets.MNIST('mnist', train=False, download=True)
images = dataset.test_data[:1000].float()
labels = dataset.test_labels[:1000]
features = images.view(1000, 784)
writer.add_embedding(features, metadata=labels, label_img=images.unsqueeze(1))
writer.add_embedding(features, global_step=1, tag='noMetadata')

In [ ]:
writer.close()